# Processing steps
## In Jupyter 

1. load ALL subject's data (pkl)
2. set fb, cc, fs, step, estimator
3. seperate Pos, Neutral, Neg or Pos, Neg
3. fcg calculate (all subject & all words of sentences & all fixation)
4. fcg stack
5. fcgs -> neural gas fit (-> mdl // encoding, symbols)
6. mdl save /// 

## In File 
7. load mdl and set property
8. load eeg & transpose EEG
9. calculate fcg
10. using mdl, encode fcg to brain_state

## Plus In Jupyter
10. encode fcgs to brain_state
11. Neg vs Pos result check (histogram, olotmatrix, markov_matrix, occupancy rate, etc.)



In [ ]:
import pickle
import os
import numpy as np
import warnings
import scipy
from dyconnmap.fc import iplv
import matplotlib.pyplot as plt
from dyconnmap import tvfcg
from dyconnmap.fc import IPLV
from dyconnmap.ts import transition_rate, occupancy_time
from dyconnmap.cluster import NeuralGas
import tqdm
import nltk

## 1. Load All subject's data

In [ ]:
# Get SR data

mat_file = []
subject = []

#rootdir = '/home/KimDY/zuco-nlp/sentiment-analysis/Data_to_preprocess/Sentence_label_data/'
rootdir = "C:/Users/김다예/OneDrive - 경희대학교/바탕 화면/연구/ZUCO/REAL/data/"
sentences = []
subject = []

for file in os.listdir(rootdir):
    if file.endswith(".pickle"):
        print("파일명 : ",file)
        subject.append(file.split("_")[2].split(".")[0])
        print("피험자 : ",file.split("_")[2].split(".")[0])

        file_name = rootdir + file
        # file load
        with open(file_name, "rb") as f:
            mat_file.append(pickle.load(f)) #저장 및 불러오기 완료

print("...done!🎉")


In [ ]:
print(subject)

## 2. set property

## 3. seperate Pos, Neutral, Neg

In [ ]:
# need to WORD LEVEL analysis and fit? 
# or Sentence LEVEL analysis and fit... << first Do
# for later word level encode... 

In [ ]:
mat_file[0][0].keys() #0 subject, 0sentence
mat_file[0][3]["RAW_EEG"].shape

In [ ]:
mat_file[0][1]["label"]
mat_file[0][2]["label_name"]
print(len(mat_file))

In [ ]:
POS_dict = []
NEU_dict = []
NEG_dict = []

for i in range(len(subject)):
    check_file = mat_file[i]
    for j in range(len(check_file)):
        label = check_file[j]["label_name"]
        if (label == 'POSITIVE'):
            POS_dict.append(mat_file[i][j])
        elif (label == 'NEUTRAL'):
            NEU_dict.append(mat_file[i][j])
        elif (label == 'NEGATIVE'):
            NEG_dict.append(mat_file[i][j])

In [ ]:
POS_eeg = []
NEU_eeg = []
NEG_eeg = []

for i in range(len(POS_dict)):
    POS_eeg.append(POS_dict[i]["RAW_EEG"].T)
    
for j in range(len(NEU_dict)):
    NEU_eeg.append(NEU_dict[j]["RAW_EEG"].T)
    
for k in range(len(NEG_dict)):
    NEG_eeg.append(NEG_dict[k]["RAW_EEG"].T)


In [ ]:
print(len(POS_dict))
print(len(NEU_dict))
print(len(NEG_dict))


print(len(POS_eeg))
print(len(NEU_eeg))
print(len(NEG_eeg))


## 4. fcg calculate

In [ ]:
# need to find more information about cc, fs etc. ex> equally seperate?

mat_file[0][0]["word_level_data"].keys()
mat_file[0][0]["word_level_data"][1] # NOTE! 1 to max (not 0)
mat_file[0][0]["word_level_data"][1].keys()

# max_sentence_length = len(mat_file[0])
# max_word_length = len(mat_file[0][0]["word_level_data"]) - 1
# because, It has 1 additional 'word_reading_order' key.

# mat_file[sub][sen].keys()
"""
'RAW_EEG', 'ICA_EEG', 'content', 'sentence_number', 'label', 
'word_embedding_idxs', 'label_content', 'label_name', 'word_level_data'
"""

# mat_file[sub][sen]["word_level_data"][word].keys()
"""
['RAW_EEG', 'ICA_EEG', 'RAW_ET', 'FFD', 'GD', 'GPT', 
'TRT', 'nFix', 'word_idx', 'content']
"""

## Sentence_fcgs

In [ ]:
"""
theta1 (4-6Hz), theta2 (6.5-8 Hz), alpha1 (8.5-10 Hz), alpha2(10.5-13 Hz), 
beta1, (13.5-18 Hz) beta2 (18.5-30Hz), 
gamma1 (30.5-40 Hz) and gamma2 (40-49.5Hz) 

"""

In [ ]:
fb = [4.0, 50.0] #lowest , Highest frequency band #band 별로 각각 살펴봐야하나?
cc = 4.0 #what is cycle-criterion ?????????
fs = 500.0 #sampling frequency
step = 250 #amount of sample that caculate once (window)

estimator = IPLV(fb, fs)
# need to (channel,time)

"""
tutorial sample : 

fb = [7.0, 13.0]
cc = 4.0
fs = 160.0
step = 80 #this time, we will slice 80 for window 

"""

In [ ]:
#check
print(POS_eeg[0].shape)
print(len(POS_dict[0]["word_level_data"])-1)

In [ ]:
# POS
i = 0
pos_err = 0
### 1_sentence >>

first_fix_eeg = POS_eeg[0]
fcg = tvfcg(np.squeeze(first_fix_eeg), estimator, fb, fs, cc, step) #1_word_1_fix
POS_fcgs = np.array(np.real(fcg))

#### and else >> 
# len(POS_eeg)
for i in tqdm.tqdm(range(1, len(POS_eeg))):
    try: 
        data = POS_eeg[i]          
        fcg = tvfcg(np.squeeze(data), estimator, fb, fs, cc, step)
        POS_fcgs = np.vstack([POS_fcgs,np.array(np.real(fcg))])

    except:
        print("window is too large for this sentence")
        pos_err +=1

print("...done!🎉")   
print("POS_fcgs = ", len(POS_fcgs))
print("Error sentence =", pos_err)


In [ ]:
print(pos_err)
POS_fcgs.shape

In [ ]:
# NEU
neu_err = 0
### 1_sentence >>
first_fix_eeg = NEU_eeg[0]
fcg = tvfcg(np.squeeze(first_fix_eeg), estimator, fb, fs, cc, step) #1_word_1_fix
NEU_fcgs = np.array(np.real(fcg))

#### and else >> 
for i in tqdm.tqdm(range(1, len(NEU_eeg))):
    try:
        data = NEU_eeg[i]          
        fcg = tvfcg(np.squeeze(data), estimator, fb, fs, cc, step)
        NEU_fcgs = np.vstack([NEU_fcgs,np.array(np.real(fcg))])
        
    except:
        print("window is too large for this sentence")
        neu_err +=1
    
print("...done!🎉")   
print("NEU_fcgs = ", len(NEU_fcgs))
print("Error sentence =", neu_err)



In [ ]:
# NEG
neg_err = 0

### 1_sentence >>
first_fix_eeg = NEG_eeg[0]
fcg = tvfcg(np.squeeze(first_fix_eeg), estimator, fb, fs, cc, step) #1_word_1_fix
NEG_fcgs = np.array(np.real(fcg))

#### and else >> 
#len(NEG_eeg)
for i in tqdm.tqdm(range(1, len(NEG_eeg))):
    try:
        data = NEG_eeg[i]         
        fcg = tvfcg(np.squeeze(data), estimator, fb, fs, cc, step)
        NEG_fcgs = np.vstack([NEG_fcgs,np.array(np.real(fcg))])

    except:
        print("window is too large for this sentence")
        neg_err +=1


print("...done!🎉")   
print("NEG_fcgs = ", len(NEG_fcgs))
print("Error sentence =", neg_err)



In [ ]:
print(POS_fcgs.shape)
#print(NEU_fcgs.shape)
print(NEG_fcgs.shape)

print(pos_err)
#print(neu_err)
print(neg_err)

## save

In [ ]:
#P/N/N dict
#P/N/N fcgs

#save_dir = "/home/KimDY/zuco-nlp/sentiment-analysis/Data_to_preprocess/"

with open("POS_theta_fcgs.pkl", "wb") as f:
   pickle.dump(POS_fcgs, f)
# with open("NEU_fcgs.pkl", "wb") as f:
#     pickle.dump(NEU_fcgs, f)
with open("NEG_theta_fcgs.pkl", "wb") as f:
    pickle.dump(NEG_fcgs, f)
    
# with open("POS_dict.pkl", "wb") as f:
#     pickle.dump(POS_dict, f)
# with open("NEU_dict.pkl", "wb") as f:
#     pickle.dump(NEU_dict, f)
# with open("NEG_dict.pkl", "wb") as f:
#     pickle.dump(NEG_dict, f)


## load

In [ ]:
import pickle
import numpy as np
import pandas as pd
import pickle
import os
import numpy as np
import warnings
import scipy
from dyconnmap.fc import iplv
import matplotlib.pyplot as plt
from dyconnmap import tvfcg
from dyconnmap.fc import IPLV
from dyconnmap.ts import transition_rate, occupancy_time
from dyconnmap.cluster import NeuralGas
import tqdm
import nltk

In [ ]:
with open("NEG_fcgs.pkl", "rb") as f:
    NEG_fcgs = pickle.load(f)
    
with open("POS_fcgs.pkl", "rb") as f:
    POS_fcgs = pickle.load(f)

In [ ]:
POS_fcgs[0].shape

## 5. fcg stack and Neural gas fit

### 2 class (binary)

In [ ]:
num_fcgs_POS, _, _ = np.shape(POS_fcgs)
num_fcgs_NEG, _, _ = np.shape(NEG_fcgs)

In [ ]:
print(POS_fcgs.shape)
print(NEG_fcgs.shape)

In [ ]:
fcgs = np.vstack([POS_fcgs, NEG_fcgs])
print(len(fcgs))
num_fcgs, num_channels, num_channels = np.shape(fcgs)

triu_ind = np.triu_indices_from(np.squeeze(fcgs[0, ...]), k=1)

fcgs = fcgs[:, triu_ind[0], triu_ind[1]]

In [ ]:
fcgs.shape

In [ ]:
rng = np.random.RandomState(0)
# what "N" is best?
mdl = NeuralGas(n_protos=10, rng=rng).fit(fcgs)


In [ ]:
encoding, symbols = mdl.encode(fcgs)

#Separate the encoded symbols based on their original groupings
grp_dist_POS = symbols[0:num_fcgs_POS]
grp_dist_NEG = symbols[num_fcgs_POS:]

In [ ]:
# save
with open("2class_mdl.pkl", "wb") as f:
    pickle.dump(mdl, f)

## CHECK BEFORE INSERT

In [1]:
#0. import 
import pickle
import numpy as np
import pandas as pd
import pickle
import os
import numpy as np
import warnings
import scipy
from dyconnmap.fc import iplv
import matplotlib.pyplot as plt
from dyconnmap import tvfcg
from dyconnmap.fc import IPLV
from dyconnmap.ts import transition_rate, occupancy_time
from dyconnmap.cluster import NeuralGas
import tqdm
import nltk

In [2]:
#7. load mdl & set property
with open("2class_mdl.pkl", "rb") as f:
    mdl = pickle.load(f)
    
fb = [4.0, 50.0] #lowest , Highest frequency band
cc = 4.0
fs = 500.0
step = 250 #this time, we will slice 80 for window 



estimator = IPLV(fb, fs)


In [3]:
#8. load EEG & Transpose EEG
with open("POS_dict.pkl", "rb") as f:
    POS_dict = pickle.load(f)    
POS_eeg = []

for i in range(len(POS_dict)):
    POS_eeg.append(POS_dict[i]["RAW_EEG"].T) #need to transpose

#POS_eeg[0].shape == (105, 3412)
    

In [5]:
#9. calculate fcg
eeg = POS_eeg[0] #1 개의 EEG 
fcg = tvfcg(np.squeeze(eeg), estimator, fb, fs, cc, step) #1_word_1_fix
fcgs = np.array(np.real(fcg))

print(fcgs.shape)
#fcgs.shape = (12, 105, 105) <- 현재 피클파일로 저장되어있는 fcg 형태. 문장에 상관없이 그냥 몽땅 Stack 되어있음. 앞으로 쓸일없을듯? 

triu_ind = np.triu_indices_from(np.squeeze(fcgs[0, ...]), k=1)
fcgs = fcgs[:, triu_ind[0], triu_ind[1]]

print(fcgs.shape)

#fcgs.shape == (12, 5460) <- encode 전에 넣어야 하는 fcg 형태 

(12, 105, 105)
(12, 5460)


In [6]:
#10. Using mdl, encode 
encoding, symbols = mdl.encode(fcgs)


In [7]:
print(symbols.shape) #일단 이게 훈련 모델에 들어감...
print(encoding.shape)

symbols #fcg 12장 각각 1개씩 있는 brain state (label, int) 정보
encoding # fcg 12장 각각에 1개씩 있는 brain state soft label 정보. 이때 soft label은 각 채널(??)마다 있음.

# 고민1. symbols 을 넣을지 encoding 을 넣을지?
# 고민2. 어떠한 형태로 넣어줄지?

(12, 1)
(12, 1, 5460)


array([[[0.06264571, 0.07973764, 0.09829245, ..., 0.04474551,
         0.02467856, 0.03130361]],

       [[0.04397912, 0.04797952, 0.05892048, ..., 0.04241011,
         0.02207803, 0.02168266]],

       [[0.04397912, 0.04797952, 0.05892048, ..., 0.04241011,
         0.02207803, 0.02168266]],

       ...,

       [[0.05965754, 0.07291722, 0.11318607, ..., 0.04981235,
         0.02560415, 0.02311476]],

       [[0.05325136, 0.06283716, 0.06421158, ..., 0.03514461,
         0.02386462, 0.02417941]],

       [[0.06264571, 0.07973764, 0.09829245, ..., 0.04474551,
         0.02467856, 0.03130361]]])

# PLUS analysis IN Jupyter_2Class

In [ ]:
grp_dist_POS = symbols[0:num_fcgs_POS]
grp_dist_NEG = symbols[num_fcgs_POS:]

In [ ]:
h_grp_dist_POS = np.histogram(grp_dist_POS, bins=mdl.n_protos, normed=True)
h_grp_dist_NEG = np.histogram(grp_dist_NEG, bins=mdl.n_protos, normed=True)

import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(12, 6))

ind = np.arange(mdl.n_protos)
p1 = ax.bar(ind - 0.125, h_grp_dist_POS[0], 0.25, label='POS')
p2 = ax.bar(ind + 0.125, h_grp_dist_NEG[0], 0.25, label='NEG')

ax.legend()
ax.set_xlabel('Symbol Index')
ax.set_ylabel('Hits %')
ax.set_xticks(np.arange(mdl.n_protos))
plt.show()

In [ ]:
protos_mtx = np.zeros((mdl.n_protos, 105, 105))

for i in range(mdl.n_protos):
    symbol_state = np.zeros((105, 105))
    symbol_state[triu_ind] = mdl.protos[i, :]
    symbol_state = symbol_state + symbol_state.T
    np.fill_diagonal(symbol_state, 1.0)
    
    protos_mtx[i, :, :] = symbol_state
    
mtx_min = np.min(protos_mtx)
mtx_max = np.max(protos_mtx)

f, ax = plt.subplots(ncols=mdl.n_protos, figsize=(12, 12))
for i in range(mdl.n_protos):
    cax = ax[i].imshow(np.squeeze(protos_mtx[i,...]), vmin=mtx_min, vmax=mtx_max, cmap=plt.cm.Spectral)
    ax[i].set_title('#{0}'.format(i))

# move the colorbar to the side ;)
f.subplots_adjust(right=0.8)
cbar_ax = f.add_axes([0.82, 0.445, 0.0125, 0.115])
cb = f.colorbar(cax, cax=cbar_ax)
cb.set_label('Imaginary PLV')

In [ ]:
print(grp_dist_POS.shape)
print(grp_dist_NEG.shape)

In [ ]:
#Examine the first subject

subj1_POS = grp_dist_POS
subj1_NEG = grp_dist_NEG

In [ ]:
from dyconnmap.ts import markov_matrix
from mpl_toolkits.axes_grid1 import ImageGrid

markov_matrix_eo = markov_matrix(subj1_POS)
markov_matrix_ec = markov_matrix(subj1_NEG)

f = plt.figure(figsize=(8, 6))
grid = ImageGrid(f, 111,
                 nrows_ncols=(1,2),
                 axes_pad=0.15,
                 share_all=True,
                 cbar_location="right",
                 cbar_mode="single",
                 cbar_size="7%",
                 cbar_pad=0.15,
                 )
im = grid[0].imshow(markov_matrix_eo, vmin=0.0, vmax=1.0, cmap=plt.cm.Spectral)
grid[0].set_xlabel('Prototype')
grid[0].set_ylabel('Prototype')
grid[0].set_title('NR')

im = grid[1].imshow(markov_matrix_ec, vmin=0.0, vmax=1.0, cmap=plt.cm.Spectral)
grid[1].set_xlabel('Prototype')
grid[1].set_ylabel('Prototype')
grid[1].set_title('TSR')

cb = grid[1].cax.colorbar(im)
cax = grid.cbar_axes[0]
axis = cax.axis[cax.orientation]
axis.label.set_text("Transition Probability")

plt.show()

In [ ]:
from dyconnmap.ts import transition_rate, occupancy_time


tr_eo = transition_rate(subj1_POS)
tr_ec = transition_rate(subj1_NEG)

print(f"""
Transition rate
===============
  NR: {tr_eo:.3f}
  TSR : {tr_ec:.3f}
""")



In [ ]:
occ_eo = occupancy_time(subj1_POS)[0]
occ_ec = occupancy_time(subj1_NEG)[0]

print("""
Occupancy time
==============
    State \t 0 \t 1 \t 2 \t 3 \t 4
      -----
  NR:      \t {0:.3f} \t {1:.3f} \t {2:.3f} \t {3:.3f} \t {4:.3f}
  TSR : \t {5:.3f} \t {6:.3f} \t {7:.3f} \t {8:.3f} \t {9:.3f}
""".format(*occ_eo, *occ_ec))

### 3 class

In [ ]:
num_fcgs_NEU, _, _ = np.shape(NEU_fcgs)

In [ ]:
fcgs = np.vstack([POS_fcgs, NEU_fcgs, NEG_fcgs]) #can?
print(len(fcgs))
num_fcgs, num_channels, num_channels = np.shape(fcgs)

triu_ind = np.triu_indices_from(np.squeeze(fcgs[0, ...]), k=1)

fcgs = fcgs[:, triu_ind[0], triu_ind[1]]

In [ ]:
rng = np.random.RandomState(0)
# what "N" is best?
mdl = NeuralGas(n_protos=5, rng=rng).fit(fcgs)


In [ ]:
# save
with open(save_dir + "3class_mdl.pkl", "wb") as f:
    pickle.dump(mdl, f)

In [ ]:
encoding, symbols = mdl.encode(fcgs)

#Separate the encoded symbols based on their original groupings
grp_dist_POS = symbols[0:num_fcgs_POS]
grp_dist_NEU = symbols[num_fcgs_POS:num_fcgs_NEU]
grp_dist_NEG = symbols[num_fcgs_NEU:]


# PLUS analysis IN Jupyter_3Class

In [ ]:
# After... 